<a href="https://colab.research.google.com/github/ovapichi/biomecanica-datos/blob/celda_de_cargas/Celda_por_equipo_Armado_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
pip install fpdf


In [37]:
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')
import fpdf as FPDF
from datetime import datetime
from fpdf import FPDF
import shutil


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
#encabezado y pie de página del pdf
class PDF (FPDF):
  def header(self):
    self.image("/content/drive/Shareddrives/Sociedad Hebraica Argentina/Imágenes/logo unsam.png",2,0.5,2.5) #Cambiar Logos
    self.image("/content/drive/Shareddrives/Sociedad Hebraica Argentina/Imágenes/logo.png",15,0.5,2.5)
    self.set_font("arial","B",12)
    self.set_text_color(50,175,175)
    self.set_font("arial","B",10)
    self.ln(12)
  def footer(self):
    self.set_y(-1)
    self.set_font("arial","",12)
    self.set_text_color(0,0,0)
    self.cell(0,1,f'{self.page_no()}',align="R")
  def set_col(self, col):
  # Set column position:
    self.col = col
    x = 0.50 + col * 2.5
    self.set_left_margin(x)
    self.set_x(x)
#Carátula del pdf
def caratula(nombre,protocolo):
  now = datetime.now()
  hoy=now.date()
  profesionales= ["Lic. Toscano, Ana Paula",'Lic. García Piccinini, Osvaldo']
  pdf.set_y(4)
  #pdf.set_text_color(50,175,175)
  pdf.set_font("arial","",12)
  pdf.image("/content/drive/Shareddrives/Sociedad Hebraica Argentina/Imágenes/informe.png",12,5,10)
  pdf.cell(0,5,f'Dirigido a: {nombre}' ,ln=2, align='l')
  pdf.multi_cell(h=2, align='l', w=10, txt=f'Protocolo: {protocolo}\n')
  pdf.cell(0,5,f'{hoy}',ln=True, align='l')
  pdf.add_page()
  pdf.set_y(4)
  pdf.set_text_color(0,0,0)
  pdf.cell(2,2,f'Equipo profesional',ln=True, align='l')
  for crack in profesionales:
    pdf.cell(2,2,f'{crack}',ln=True, align='l')
  #pdf.image("/content/drive/Shareddrives/Sociedad Hebraica Argentina/Imágenes/firmas lab.jpg",5.5,16,10)
  pdf.set_font("arial","",10)
  pdf.multi_cell(h=0.5, align='L', w=0, txt='El equipo responsable del Laboratorio de Biomecánica Análisis Funcional del Movimiento y Rendimiento Humano queda a su disposición para el análisis conjunto del caso, su interpretación, tanto como las eventuales consideraciones de las conclusiones. \nAgradecemos la confianza al permitirnos realizar esta evaluación. \nLo saludamos cordialmente.')
  return
  # =================================BLOQUE DE FUNCIONES=================================================



In [10]:

  #CELDA DE CARGAS WIN LABORAT
  #INGRESAR CARPETA DE ARCHIVOS .dat
direccion_crudos="/content/drive/Shareddrives/Sociedad Hebraica Argentina/Noviembre 2023/Datos/Celda/Crudos/"
direccion_final="/content/drive/Shareddrives/Sociedad Hebraica Argentina/Noviembre 2023/Datos/Celda/"
fecha="Noviembre 2023"
df_final = pd.DataFrame()
df_individual = pd.DataFrame(index=[0],columns='cuad_der cuad_izq isq_der isq_izq'.split())
lista=[]
#Entrega un .csv con formato de columnas ["jugadora","maximo","musculo","tiempo"]

  def leer_carpeta(direccion):
    lista_archivos=[]
    for nombre_archivo in os.listdir(direccion):
      lista_archivos.append(nombre_archivo)
    lista_archivos.sort()
    #print(lista_archivos)
    return lista_archivos

  def seleccion_muslo(lista,direccion):
    jugadora=[]
    maximos=[]
    musculo=[]

    for archivo in lista:

      df=pd.read_csv(direccion+"/"+archivo,on_bad_lines='skip',sep = "\t", decimal=",",header=None, encoding="utf-16")
      df=df.drop([0],axis=0)
      df=df.drop([0,1,2], axis=1)
      max=float(df[3].max())
      maximos.append(max)
      #print(max)

      if archivo[-12:-4]=="Cuad Der":
        musculo.append("Cuadriceps Derechos")
        jugadora.append(archivo[:-13])
        df_individual['cuad_der'][0]=max
      elif archivo[-12:-4]=="Cuad Izq":
        musculo.append("Cuadriceps Izquierdos")
        jugadora.append(archivo[:-13])
        df_individual['cuad_izq'][0]=max
      elif archivo[-12:-4]==" Isq Der":
        jugadora.append(archivo[:-12])
        musculo.append("Isquios Derechos")
        df_individual['isq_der'][0]=max
      elif archivo[-12:-4]==" Isq Izq":
        jugadora.append(archivo[:-12])
        musculo.append("Isquios Izquierdo")
        df_individual['isq_izq'][0]=max
        df_individual.to_csv(direccion_final+"Procesados/"+archivo[:-12]+".csv")


    df_final['jugadora']=jugadora
    df_final['maximos']=maximos
    df_final['musculo']=musculo
    df_final['tiempo']=5
    return




lista=leer_carpeta(direccion_crudos)
seleccion_muslo(lista,direccion_crudos)

df_final.to_csv(direccion_final+"Celda"+fecha+".csv")
df_final.dtypes

jugadora     object
maximos     float64
musculo      object
tiempo        int64
dtype: object

In [69]:

#función para la aplicación de fórmulas de asimetría


def procesar_maximos(cuad_der,cuad_izq,isq_der,isq_izq,nombre):

    relacionFlex=(isq_der/isq_izq)*100-100
    simetricoFlex=0
    asimetricoDFlex=0
    asimetricoIFlex=0
    simetricoFlex=(relacionFlex <=10) & (relacionFlex >=-10)
    asimetricoDFlex=(relacionFlex <-10) & (relacionFlex >=-25)
    asimetricoIFlex=(relacionFlex >10) & (relacionFlex <25)
    #relacion extensores
    simetricoExt=0
    asimetricoDExt=0
    asimetricoIExt=0

    relacionExt=(cuad_der/cuad_izq)*100-100

    simetricoExt=(relacionExt <=10) & (relacionExt >=-10)
    asimetricoDExt=(relacionExt <-10) & (relacionExt >=-25)
    asimetricoIExt=(relacionExt >10) & (relacionExt <25)

    #relacion interpierna derecha
    criticoD=(cuad_der+isq_der)*0.06
    sCriticoD=(cuad_der+isq_der)*0.3
    relacionMd=(cuad_der+isq_der)*0.7-cuad_der

    simetricoMd=0
    asimetricoDIsq=0
    asimetricoDCuad=0
    sAsimetricoDIsq=0
    sAsimetricoDCuad=0

    simetricoMd=(relacionMd <=criticoD) & (relacionMd >=criticoD*-1)
    asimetricoDIsq=(relacionMd <criticoD*-1) & (relacionMd >=sCriticoD*-1)
    asimetricoDCuad=(relacionMd >criticoD) & (relacionMd <sCriticoD)
    sAsimetricoDIsq=(relacionMd <=sCriticoD*-1)
    sAsimetricoDCuad=(relacionMd >= sCriticoD)

    #relacion interpierna Izquierda
    criticoI=(cuad_izq+isq_izq)*0.06
    sCriticoI=(cuad_izq+isq_izq)*0.3
    relacionMi=(cuad_izq+isq_izq)*0.7-cuad_izq

    simetricoMi=0
    asimetricoIIsq=0
    asimetricoICuad=0
    sAsimetricoIIsq=0
    sAsimetricoICuad=0

    simetricoMi=(relacionMi <=criticoI) & (relacionMi >=criticoI*-1)
    asimetricoIIsq=(relacionMi <criticoI*-1) & (relacionMi >=sCriticoI*-1)
    asimetricoICuad=(relacionMi >criticoI) & (relacionMi <sCriticoI)
    sAsimetricoIIsq=(relacionMi <=sCriticoI*-1)
    sAsimetricoICuad=(relacionMi >= sCriticoI)

    pdf.add_page()
    pdf.set_font("Arial",size=12)
    caratula("Club Hebraica Hockey", "Evaluación de la fuerza isométrica de MMII con celda de cargas")
    pdf.add_page()
    pdf.set_font("Arial",size=10, style="B")
    pdf.set_y(2)

    pdf.multi_cell(h=0.5, align='L', w=0, txt=f"RESULTADOS DE LA EVALUACIÓN CON CELDAS DE CARGA \n \n1) FUERZA DE MUSLOS\n \n1.1) RELACION FLEXORES \nValores de referencia \n(-10(ok)<10-25(leve)<+25(grave) \n")
    pdf.cell(2,2,f'Porcentaje de asiemtría ={relacionFlex:.2f}')
    if simetricoFlex !=0:
      pdf.cell(2,4,f'Valores Óptimos en Flexores',ln=False, align='l')
    elif asimetricoDFlex !=0:
      pdf.cell(2,4,f'Fortalecer Flexores lado Derecho',ln=False, align='l')
    else:
      pdf.cell(2,4,f'Fortalecer Flexores lado Izquierdo',ln=False, align='l')


    pdf.set_y(8)

    pdf.multi_cell(h=0.5, align='L', w=0, txt=f"2.2) RELACION EXTENSORES \nValores de referencia \n(-10(ok)<10-25(leve)<+25(grave))\n")
    pdf.cell(2,2,f'Porcentaje de asiemtría ={relacionExt:.2f}')

    if simetricoExt !=0:
      pdf.cell(2,4,f'Valores Óptimos en Extensores')
    elif asimetricoDExt !=0:
      pdf.cell(2,4,f'Fortalecer Extensores lado Derecho')
    else:
      pdf.cell(2,4,f'Fortalecer Extensores lado Izquierdo')

    pdf.set_y(12)

    pdf.multi_cell(h=0.5, align='L', w=0, txt=f"2.3) RELACION MUSLO DERECHO \nValores de referencia \n(-,{criticoD:.2f},(ok)<,{criticoD:.2f},-,{sCriticoD:.2f},(leve)<+,{sCriticoD:.2f},(grave))\n")
    pdf.cell(2,2,f'Porcentaje de relación ={relacionMd:.2f}')

    if simetricoMd !=0:
      pdf.cell(2,4,f'Valores Óptimos en Muslo Derecho')
    elif asimetricoDIsq !=0 or sAsimetricoDIsq!=0:
      pdf.cell(2,4,f'Fortalecer Flexores lado Derecho')
    else:
      pdf.cell(2,4,f'Fortalecer Extensores lado Derecho')

    pdf.set_y(16)

    pdf.multi_cell(h=0.5, align='L', w=0, txt=f"2.4) RELACION MUSLO Izquierdo \nValores de referencia \n(-{criticoI:.2f},(ok)<,{criticoI:.2f},-{sCriticoI:.2f},(leve)<+{sCriticoI:.2f},(grave))\n")
    pdf.cell(2,2,f'Porcentaje de relación ={relacionMi:.2f}')

    if simetricoMi !=0:
      pdf.cell(2,4,f'Valores Óptimos en Muslo Izquierdo')
    elif asimetricoIIsq !=0 or sAsimetricoIIsq!=0:
      pdf.cell(2,4,f'Fortalecer Flexores lado Izquierdo')
    else:
      pdf.cell(2,4,f'Fortalecer Extensores lado Izquierdo')
    #print(nombre)
    exportar(nombre)

    return


def exportar(nombre):
  #print(nombre)
  pdfFileName = nombre +'.pdf'#NO EXPORTA EL PDF PROBLEMA EN LOS ARCHIVOS
  pdf.output(pdfFileName)

  shutil.copy(pdfFileName,"/content/drive/Shareddrives/Sociedad Hebraica Argentina/Noviembre 2023/Datos/Celda/Informes")
  return
lista=leer_carpeta(direccion_final+"Procesados/")
#print(lista)
for archivo in lista:
  df2=pd.read_csv(direccion_final+"Procesados/"+archivo,decimal=".")
  df2=df2.drop(["Unnamed: 0"],axis=1)
  #print(df2.dtypes)

  #print(df2)
  cd=float(df2['cuad_der'][0])
  ci=float(df2['cuad_izq'][0])
  id=float(df2['isq_der'][0])
  ii=float(df2['isq_izq'][0])
  name=archivo[:-4]
  #print(cd,ci,id,ii)
  pdf = PDF("p","cm","A4")

  procesar_maximos(cd,ci,id,ii,name)